# Analyzing the different regions and their correlations

Useful links:

* https://en.wikibooks.org/wiki/Introducing_Julia/DataFrames

In [89]:
# Packages we will use throughout this notebook (only first time)
Pkg.add("UMAP")
Pkg.add("Makie")
Pkg.add("XLSX")
Pkg.add("VegaDatasets")
Pkg.add("MultivariateStats")
Pkg.add("RDatasets")
Pkg.add("StatsBase")
Pkg.add("Statistics")
Pkg.add("LinearAlgebra")
Pkg.add("Plots")
Pkg.add("ScikitLearn")
Pkg.add("MLBase")
Pkg.add("Distances")

   Resolving package versions...


LoadError: InterruptException:

In [90]:
# Packages we will use throughout this notebook
using UMAP
using Makie
using XLSX
using VegaDatasets
using DataFrames
using MultivariateStats
using RDatasets
using StatsBase
using Statistics
using LinearAlgebra
using Plots
using ScikitLearn
using MLBase
using Distances

In [ ]:
#using PlotlyJS

In [2]:
using DataFrames

In [3]:
using CSV

In [91]:
features = DataFrame(CSV.File("data/Kidney_Sample_Annotations.txt"))

,SlideName,ScanName,ROILabel,SegmentLabel,SegmentDisplayName
,String,String,Int64,String,String
1,disease3,disease3_scan,7,Geometric Segment,disease3_scan | 007 | Geometric Segment
2,disease3,disease3_scan,8,Geometric Segment,disease3_scan | 008 | Geometric Segment
3,disease3,disease3_scan,9,Geometric Segment,disease3_scan | 009 | Geometric Segment
4,disease3,disease3_scan,10,Geometric Segment,disease3_scan | 010 | Geometric Segment
5,disease3,disease3_scan,11,Geometric Segment,disease3_scan | 011 | Geometric Segment
6,disease3,disease3_scan,12,Geometric Segment,disease3_scan | 012 | Geometric Segment
7,disease3,disease3_scan,13,Geometric Segment,disease3_scan | 013 | Geometric Segment
8,disease3,disease3_scan,14,Geometric Segment,disease3_scan | 014 | Geometric Segment
9,disease3,disease3_scan,15,Geometric Segment,disease3_scan | 015 | Geometric Segment


In [92]:
names(features)

25-element Vector{String}:
 "SlideName"
 "ScanName"
 "ROILabel"
 "SegmentLabel"
 "SegmentDisplayName"
 "Sample_ID"
 "AOISurfaceArea"
 "AOINucleiCount"
 "ROICoordinateX"
 "ROICoordinateY"
 "RawReads"
 "TrimmedReads"
 "StitchedReads"
 "AlignedReads"
 "DeduplicatedReads"
 "SequencingSaturation"
 "UMIQ30"
 "RTSQ30"
 "disease_status"
 "pathology"
 "region"
 "LOQ"
 "NormalizationFactor"
 "RoiReportX"
 "RoiReportY"

In [94]:
unique(features.ScanName)

7-element Vector{String}:
 "disease3_scan"
 "disease4_scan"
 "normal3_scan"
 "normal4_scan"
 "disease1B_scan"
 "disease2B_scan"
 "normal2B_scan"

In [95]:
replace("DSP.1001250007851.H.A02","."=>"-") in features.Sample_ID

true

In [96]:
CellTypes = DataFrame(CSV.File("data/Cell_Types_for_Spatial_Decon.txt"))

,ClusterID,Alias,Data_set
,String,String,String
1,N0,H,barcodeLevelGroup_normalEpitheliumAndVascularWithoutProximalTubularV2
2,N1,N1,barcodeLevelGroup_normalEpitheliumAndVascularWithoutProximalTubularV2
3,N2,AV1,barcodeLevelGroup_normalEpitheliumAndVascularWithoutProximalTubularV2
4,N3,N3,barcodeLevelGroup_normalEpitheliumAndVascularWithoutProximalTubularV2
5,N4,DV,barcodeLevelGroup_normalEpitheliumAndVascularWithoutProximalTubularV2
6,N5,N5,barcodeLevelGroup_normalEpitheliumAndVascularWithoutProximalTubularV2
7,N6,N6,barcodeLevelGroup_normalEpitheliumAndVascularWithoutProximalTubularV2
8,N7,GE,barcodeLevelGroup_normalEpitheliumAndVascularWithoutProximalTubularV2
9,N8,N8,barcodeLevelGroup_normalEpitheliumAndVascularWithoutProximalTubularV2


In [97]:
names(CellTypes)

10-element Vector{String}:
 "ClusterID"
 "Alias"
 "Data_set"
 "Number_of_cells"
 "Cell_type1"
 "Cell_type2"
 "Cell_type3"
 "cell_type_specific"
 "cell_type_general"
 "cluster_name"

In [70]:
unique(CellTypes.ClusterID)

35-element Vector{String}:
 "N0"
 "N1"
 "N2"
 "N3"
 "N4"
 "N5"
 "N6"
 "N7"
 "N8"
 "N9"
 "N11"
 "N12"
 "N13"
 ⋮
 "IN5"
 "IN6"
 "IN7"
 "IN8"
 "IN9"
 "IN10"
 "IN11"
 "IN12"
 "IN13"
 "IN14"
 "IN15"
 "IN16"

In [98]:
CDC = DataFrame(CSV.File("data/Kidney_Spatial_Decon.txt"))

,Alias,DSP.1001250007851.H.A02,DSP.1001250007851.H.A03,DSP.1001250007851.H.A04
,String,Float64,Float64,Float64
1,8T,0.0,0.0,0.0
2,AV1,4.5904,4.06622,5.24017
3,AV2,1.21088,0.430753,1.38894
4,B,0.135515,0.37214,0.386018
5,C1,0.0301434,0.0,0.0
6,C2,0.0,0.0,0.0
7,D,0.0132459,0.0,0.0
8,DV,3.56322,2.57307,3.92167
9,G,1.68707,4.0284,3.15502


In [100]:
TCM = DataFrame(CSV.File("data/Kidney_Raw_TargetCountMatrix.txt"))

,TargetName,disease3_scan | 001 | PanCK,disease3_scan | 001 | neg,disease3_scan | 002 | PanCK
,String,Float64,Float64,Float64
1,PADI2,15.0,35.0,36.0
2,CYP24A1,23.0,32.0,23.0
3,SUPT16H,35.0,66.0,46.0
4,ZMIZ2,51.0,81.0,87.0
5,SPAG9,48.0,65.0,46.0
6,AMELX,21.0,30.0,31.0
7,VPS33B,44.0,55.0,32.0
8,DDX58,29.0,48.0,49.0
9,MAK,25.0,40.0,28.0


In [7]:
PCM = DataFrame(CSV.File("data/Kidney_Q3Norm_TargetCountMatrix.txt"))

,TargetName,disease3_scan | 001 | PanCK,disease3_scan | 001 | neg,disease3_scan | 002 | PanCK
,String,Float64,Float64,Float64
1,PADI2,6.01291,8.6185,11.7082
2,CYP24A1,9.2198,7.87978,7.48021
3,SUPT16H,14.0301,16.252,14.9604
4,ZMIZ2,20.4439,19.9457,28.2947
5,SPAG9,19.2413,16.0058,14.9604
6,AMELX,8.41807,7.38729,10.082
7,VPS33B,17.6379,13.5434,10.4073
8,DDX58,11.625,11.8197,15.9361
9,CDV3,22.0473,19.6994,20.164


In [8]:
M = Matrix(PCM[:,2:end])

16084×231 Matrix{Float64}:
  6.01291   8.6185   11.7082   …   4.30925  17.237    8.6185   8.6185
  9.2198    7.87978   7.48021     12.9278    8.6185  25.8555   8.6185
 14.0301   16.252    14.9604       4.30925  17.237    8.6185   8.6185
 20.4439   19.9457   28.2947       8.6185   17.237   25.8555  17.237
 19.2413   16.0058   14.9604      21.5463   34.474   51.711   17.237
  8.41807   7.38729  10.082    …   4.30925  17.237   17.237    8.6185
 17.6379   13.5434   10.4073       4.30925   8.6185   8.6185   8.6185
 11.625    11.8197   15.9361      12.9278    8.6185  25.8555   8.6185
 22.0473   19.6994   20.164        8.6185   25.8555  60.3295  51.711
 14.8318   13.2971   14.9604       4.30925  17.237    8.6185  17.237
 14.431    15.5133   14.9604   …  12.9278   17.237    8.6185   8.6185
 18.8405   17.4833   20.164       17.237     8.6185   8.6185   8.6185
 11.625    10.5884   10.4073       4.30925   8.6185  17.237    8.6185
  ⋮                            ⋱                               ⋮
 1

In [9]:
data = M
data = (data .- mean(data,dims = 2))./ std(data,dims=2)

16084×231 Matrix{Float64}:
 -0.572635     0.130916    0.96517    …   0.130916      0.130916
  0.0609006   -0.119159   -0.172848       2.29625      -0.0198952
 -0.390533     0.0750558  -0.195594      -1.52451      -1.52451
 -0.419471    -0.495582    0.779878       0.407248     -0.909379
 -0.983715    -1.30961    -1.4149         2.28673      -1.18559
  0.00936344  -0.295399    0.501327   …   2.61677       0.0686227
  1.60162      0.571413   -0.217658      -0.66772      -0.66772
 -0.653512    -0.614887    0.201727       2.16953      -1.24993
 -0.693934    -0.931553   -0.884533       3.18043       2.30819
 -0.00624055  -0.358182    0.0232448     -1.43108       0.545309
 -0.425986    -0.227344   -0.328817   …  -1.49276      -1.49276
  0.0135777   -0.187312    0.209494      -1.49945      -1.49945
  0.873615     0.572578    0.519952       2.50354       0.000440403
  ⋮                                   ⋱                 ⋮
  2.63243      2.17797     0.47894       -0.000626416  -0.000626416
 -0.

In [ ]:
p = fit(PCA,data,maxoutdim=2)

In [ ]:
P = projection(p)

In [ ]:
Yte = MultivariateStats.transform(p, data) #notice that Yte[:,1] is the same as P'*(data[1,:]-mean(p))

In [ ]:
   patientState = ["DKD", "normal"]
xaxis_type = "Linear"
yaxis_type = "Linear"
 PlotlyJS.plot(
        M[1,features[!, "disease_status"] .== patientState[1]],
        M[2,features[!, "disease_status"] .== patientState[1]],
        Layout(
            xaxis_type = xaxis_type == "Linear" ? "linear" : "log",
            xaxis_title = "pca_1",
            yaxis_title = "pca_2",
            yaxis_type = yaxis_type == "Linear" ? "linear" : "log",
            hovermode = "closest",
            height = 450,
        ),
        kind = "scatter",
        mode = "markers",
        marker_size = 15,
        marker_opacity = 0.5,
        marker_line_width = 0.5,
        marker_line_color = "white",

    )

In [ ]:
PlotlyJS.plot(
        Yte[1,features[!, "disease_status"] .== patientState[1]],
        Yte[2,features[!, "disease_status"] .== patientState[1]],
        Layout(
            xaxis_type = xaxis_type == "Linear" ? "linear" : "log",
            xaxis_title = "pca_1",
            yaxis_title = "pca_2",
            yaxis_type = yaxis_type == "Linear" ? "linear" : "log",
            hovermode = "closest",
            height = 450,
        ),
        kind = "scatter",
        mode = "markers",
        marker_size = 15,
        marker_opacity = 0.5,
        marker_line_width = 0.5,
        marker_line_color = "white",
)

In [ ]:
features.segmentDi

In [ ]:
#To plot we need to know which points are what...
#Preliminarily, we can use this:
segmentDisplayNames=features[!,r"SegmentDi."]

In [ ]:
healthy=[!occursin(r"disease.",seg) for seg in segmentDisplayNames[!,1]];

In [ ]:
glomeruli = [occursin(r".Geo.",seg) for seg in segmentDisplayNames[!,1]];
distTub = [occursin(r".Pan.",seg) for seg in segmentDisplayNames[!,1]];
proxTub = [occursin(r".neg",seg) for seg in segmentDisplayNames[!,1]];

In [ ]:
Plots.scatter(Yte[1,healthy.& glomeruli],Yte[2,healthy.& glomeruli],color=1,label="Glomeruli (Healthy)",marker=:o)
Plots.xlabel!("pca component1")
Plots.ylabel!("pca component2")
Plots.scatter!(Yte[1,.!(healthy).& glomeruli],Yte[2,.!(healthy).& glomeruli],color=1,label="Glom (DKD)",marker=:d)
Plots.scatter!(Yte[1,healthy.& distTub],Yte[2,healthy.& distTub],color=2,label="Distub",marker=:o)
Plots.scatter!(Yte[1,.!(healthy).& distTub],Yte[2,.!(healthy).& distTub],color=2,label=:none, marker=:d)
Plots.scatter!(Yte[1,healthy.& proxTub],Yte[2,healthy.& proxTub],color=3,label="Proxtub",marker=:o)
Plots.scatter!(Yte[1,.!(healthy).& proxTub],Yte[2,.!(healthy).& proxTub],color=3,label=:none, marker=:d)

In [ ]:
a=[stat for stat in unique(features[!,"disease_status"])]

In [ ]:
@show a

In [ ]:
Plots.png("glomAndTubs.png")

In [ ]:
glomPath = features[!,"pathology"];
glomPath = [ ismissing(x) ? "a" : x for x in glomPath ]

In [ ]:
features.SegmentDisplayName

In [ ]:
Plots.scatter(Yte[1,glomPath.=="healthy"],Yte[2,glomPath.=="healthy"],color=1,label="healthy Glomeruli",marker=:o)
Plots.xlabel!("pca component1")
Plots.ylabel!("pca component2")
Plots.scatter!(Yte[1,glomPath.=="abnormal"],Yte[2,glomPath.=="abnormal"],color=1,label="abnormal Glom",marker=:x)


In [ ]:
Plots.scatter(Yte[1, .!(healthy) .& (glomPath.=="healthy")],Yte[2,.!(healthy) .& (glomPath.=="healthy")],color=1,label="healthy Glomeruli(DKD)",marker=:o)
Plots.scatter!(Yte[1,.!(healthy) .& (glomPath.=="abnormal")],Yte[2,.!(healthy) .& (glomPath.=="abnormal")],color=1,label="abnormal Glom(DKD)",marker=:x)

Plots.scatter!(Yte[1, (healthy) .& (glomPath.=="healthy")],Yte[2,(healthy) .& (glomPath.=="healthy")],color=2,label="healthy Glomeruli(Ctrl)",marker=:o)
Plots.xlabel!("pca component1")
Plots.ylabel!("pca component2")
Plots.scatter!(Yte[1,(healthy) .& (glomPath.=="abnormal")],Yte[2,(healthy) .& (glomPath.=="abnormal")],color=2,label="abnormal Glom(Ctrl)",marker=:x)


In [ ]:
p = fit(PCA,data,maxoutdim=3)
Yte = MultivariateStats.transform(p, data)
scatter3d(Yte[1,:],Yte[2,:],Yte[3,:],legend=false)

In [ ]:
scene = Makie.scatter(Yte[1,:],Yte[2,:],Yte[3,:])

In [ ]:
display(scene)

In [ ]:
car_origin = C[:,:Origin]
carmap = labelmap(car_origin) #from MLBase
uniqueids = labelencode(carmap,car_origin)

In [ ]:
Plots.png("glomeruliDKDvsCtrl.png")

In [11]:
patients=unique(features[!,"SlideName"])

7-element Vector{String}:
 "disease3"
 "disease4"
 "normal3"
 "normal4"
 "disease1B"
 "disease2B"
 "normal2B"

In [ ]:


using AbstractPlotting 

img = rand(100, 100)
scene = Scene(resolution = (500, 500))
AbstractPlotting.heatmap!(scene, img, scale_plot = false)
clicks = Node(Point2f0[(0,0)])
on(scene.events.mousebuttons) do buttons
   if AbstractPlotting.ispressed(scene, Mouse.left)
       pos = AbstractPlotting.to_world(scene, AbstractPlotting.Point2f0(scene.events.mouseposition[]))
       clicks[] = push!(clicks[], pos)
   end
   return
end
AbstractPlotting.scatter!(scene, clicks, color = :red, marker = '+', markersize = 10)


In [ ]:
patient = patients[1]
patientRegions= features[!,"SlideName"].==patient
healthyGlom = glomPath.=="healthy"
abnormalGlom = glomPath.=="abnormal"
Plots.scatter(Yte[1,patientRegions .& healthyGlom],Yte[2,patientRegions .& healthyGlom],color=1,label="healthy Glomeruli",marker=:o)
Plots.title!(patient)
Plots.xlabel!("pca component1")
Plots.ylabel!("pca component2")
Plots.scatter!(Yte[1,patientRegions .& abnormalGlom],Yte[2,patientRegions .& abnormalGlom],color=1,label="abnormal Glom",marker=:x)

In [ ]:
inspectdr()

In [ ]:
patient = patients[1]
patientRegions= features[!,"SlideName"].==patient
healthyGlom = glomPath.=="healthy"
abnormalGlom = glomPath.=="abnormal"
Plots.scatter(Yte[1,patientRegions .& healthyGlom],Yte[2,patientRegions .& healthyGlom],color=1,label="healthy Glomeruli",marker=:o)
Plots.title!(patient)
Plots.xlabel!("pca component1")
Plots.ylabel!("pca component2")
Plots.scatter!(Yte[1,patientRegions .& abnormalGlom],Yte[2,patientRegions .& abnormalGlom],color=1,label="abnormal Glom",marker=:x)

In [ ]:
plotly()

In [ ]:
Pkg.add.(["Dash", "DashCoreComponents", "DashHtmlComponents", "DashTable"])

In [ ]:
using Pkg

In [ ]:
patient = patients[1]
patientRegions= features[!,"SlideName"].==patient
healthyGlom = glomPath.=="healthy"
abnormalGlom = glomPath.=="abnormal"
Plots.scatter(Yte[1,patientRegions .& healthyGlom],Yte[2,patientRegions .& healthyGlom],color=1,label="healthy Glomeruli",marker=:o)
Plots.title!(patient)
Plots.xlabel!("pca component1")
Plots.ylabel!("pca component2")
b=Plots.scatter!(Yte[1,patientRegions .& abnormalGlom],Yte[2,patientRegions .& abnormalGlom],color=1,label="abnormal Glom",marker=:x)

In [ ]:
patient = patients[1]
patientRegions= features[!,"SlideName"].==patient
healthyGlom = glomPath.=="healthy"
abnormalGlom = glomPath.=="abnormal"
Plots.scatter(Yte[1,patientRegions .& healthyGlom],Yte[2,patientRegions .& healthyGlom],color=1,label="healthy Glomeruli",marker=:o)
Plots.title!(patient)
Plots.xlabel!("pca component1")
Plots.ylabel!("pca component2")
Plots.scatter!(Yte[1,patientRegions .& abnormalGlom],Yte[2,patientRegions .& abnormalGlom],color=1,label="abnormal Glom",marker=:x)

In [ ]:
features[patientRegions,:]

In [ ]:
#Patient by Patient plotting
for (num,patient) in enumerate(patients)
    patientRegions= features[!,"SlideName"].==patient
    healthyGlom = glomPath.=="healthy"
    abnormalGlom = glomPath.=="abnormal"
    Plots.scatter!(Yte[1,patientRegions .& healthyGlom],Yte[2,patientRegions .& healthyGlom],color=num,marker=:o,legend=false)
    Plots.title!(patient)
    Plots.xlabel!("pca component1")
    Plots.ylabel!("pca component2")
    Plots.scatter!(Yte[1,patientRegions .& abnormalGlom],Yte[2,patientRegions .& abnormalGlom],color=num,marker=:x)
    Plots.png("test.png")
end


In [ ]:
names(PCM)

In [ ]:
Plots.plot(PCM[!,"normal2B_scan | 008 | PanCK"])

In [24]:
genes=Vector(PCM[1,points[gloms,"SegmentDisplayName"]])

47-element Vector{Float64}:
  5.560325635
  4.104049873
  7.660893097
  8.618504734
  5.417345833
  5.386565459
  9.15716128
 10.13941733
  7.584284166
  8.340488452
  7.097592134
  9.979321271
  7.097592134
  ⋮
 10.77313092
  2.462429924
  8.164899222
  8.618504734
  8.618504734
  7.097592134
 13.31950732
  8.618504734
  8.618504734
  8.208099747
  7.051503873
  3.0418252

In [34]:
using Pkg
Pkg.add("PlotlyBase")

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
    Updating `~/.julia/environments/v1.6/Project.toml`
  [a03496cd] + PlotlyBase v0.4.3
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [39]:
selectedGene = PCM.TargetName[1]

"PADI2"

In [47]:
ormalizedCounts = Vector(PCM[PCM[!,"TargetName" ] .== selectedGene,points[gloms,:].SegmentDisplayName][1,:])

47-element Vector{Float64}:
  5.560325635
  4.104049873
  7.660893097
  8.618504734
  5.417345833
  5.386565459
  9.15716128
 10.13941733
  7.584284166
  8.340488452
  7.097592134
  9.979321271
  7.097592134
  ⋮
 10.77313092
  2.462429924
  8.164899222
  8.618504734
  8.618504734
  7.097592134
 13.31950732
  8.618504734
  8.618504734
  8.208099747
  7.051503873
  3.0418252

In [35]:
patientid=1
patient=patients[patientid]
patientRegions= features[!,"SlideName"].==patient
points = features[patientRegions,["SegmentDisplayName", "ROICoordinateX", "ROICoordinateY"]]
gloms=[occursin(r"Geom.",x) for x in points[!,"SegmentDisplayName"]]
Plots.scatter(points.ROICoordinateX,points.ROICoordinateY,color=1,label="Tubules")
Plots.title!(patient)
Plots.scatter!(points[gloms,:].ROICoordinateX,points[gloms,:].ROICoordinateY,color=2,label="Gloms")

<!DOCTYPE html>
 
 
 Plots.jl

In [32]:
plotlyjs()

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-4422728411971066278\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-13165516602854216741\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-4422728411971066278\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Plots.PlotlyJSBackend()

In [101]:
structuresDict = Dict("abnormal"=>"Glom (Abnormal)","healthy"=>"Glom (Healthy)"," PanCK" => "Tub. Distal", " neg" => "Tub. Proximal")
comprehensiveStates = [structuresDict[ismissing(row.pathology) ? split(row["SegmentDisplayName"],"|")[3] : row.pathology] for row in eachrow(features)]

231-element Vector{String}:
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 ⋮
 "Tub. Distal"
 "Tub. Proximal"
 "Tub. Distal"
 "Tub. Proximal"
 "Tub. Distal"
 "Tub. Proximal"
 "Tub. Distal"
 "Tub. Proximal"
 "Tub. Distal"
 "Tub. Proximal"
 "Glom (Healthy)"
 "Glom (Healthy)"

In [112]:
unique(features.disease_status)

2-element Vector{String}:
 "DKD"
 "normal"

In [115]:
a=[1,2,3];b=[3,5]

2-element Vector{Int64}:
 3
 5

In [116]:
[(x,y) for x in a for y in b]

6-element Vector{Tuple{Int64, Int64}}:
 (1, 3)
 (1, 5)
 (2, 3)
 (2, 5)
 (3, 3)
 (3, 5)

In [ ]:
status=="normal" ? "circle" : "square"

In [108]:
insertcols!(features,"states"=>comprehensiveStates)

,SlideName,ScanName,ROILabel,SegmentLabel,SegmentDisplayName
,String,String,Int64,String,String
1,disease3,disease3_scan,7,Geometric Segment,disease3_scan | 007 | Geometric Segment
2,disease3,disease3_scan,8,Geometric Segment,disease3_scan | 008 | Geometric Segment
3,disease3,disease3_scan,9,Geometric Segment,disease3_scan | 009 | Geometric Segment
4,disease3,disease3_scan,10,Geometric Segment,disease3_scan | 010 | Geometric Segment
5,disease3,disease3_scan,11,Geometric Segment,disease3_scan | 011 | Geometric Segment
6,disease3,disease3_scan,12,Geometric Segment,disease3_scan | 012 | Geometric Segment
7,disease3,disease3_scan,13,Geometric Segment,disease3_scan | 013 | Geometric Segment
8,disease3,disease3_scan,14,Geometric Segment,disease3_scan | 014 | Geometric Segment
9,disease3,disease3_scan,15,Geometric Segment,disease3_scan | 015 | Geometric Segment


In [109]:
features.states

231-element Vector{String}:
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 "Glom (Abnormal)"
 ⋮
 "Tub. Distal"
 "Tub. Proximal"
 "Tub. Distal"
 "Tub. Proximal"
 "Tub. Distal"
 "Tub. Proximal"
 "Tub. Distal"
 "Tub. Proximal"
 "Tub. Distal"
 "Tub. Proximal"
 "Glom (Healthy)"
 "Glom (Healthy)"

In [33]:

Plots.plot(points[gloms,:].ROICoordinateX,points[gloms,:].ROICoordinateY,color=2,label="Gloms")

<!DOCTYPE html>
 
 
 Plots.jl

In [ ]:
modePerRegion=[argmax(PCM[!,nn]) for nn in names(PCM)];

In [ ]:
Plots.histogram(modePerRegion)

# GSEA

In [ ]:
ssGSEA = DataFrame(CSV.File("data/Kidney_ssGSEA.txt"))

In [ ]:
names(ssGSEA)

In [ ]:
groupPerRegion=[ssGSEA[argmax(ssGSEA[!,nn]),"Column1"] for nn in names(ssGSEA)];

In [ ]:
groupPerRegion

In [ ]:
s=unique(groupPerRegion);

In [ ]:
datamap = countmap(groupPerRegion)
bar((x -> datamap[x]).(s), yticks=(1:length(s), s),orientation = :horizontal,yflip=true,size=(1200,500))

In [ ]:
png("GEA_histogram.png")